In [ ]:
! rm -rf ../output/*

In [ ]:
import os
os.chdir(os.getcwd() + "/..")

In [ ]:
from src.jobcontext import JobContext
from pyhocon import ConfigFactory

class Args:
    start = "2023-06-01"
    end = "2023-06-03"
    job = "SimpleJob"
    config = "resource/config.conf"

args = Args()

config = ConfigFactory.parse_file(args.config)
context = JobContext(config, args.job, args.start, args.end)
spark = context.create_spark_session()
context.spark = spark

In [ ]:
from src.jobs import SimpleJob

job = SimpleJob(context)
data = job.transform()

In [ ]:
pandas_df = data.toPandas()

In [ ]:
pandas_df

In [ ]:
from src.datatypes.btcusdt_trades import BtcUsdtTrades
import pandas as pd
datatype = BtcUsdtTrades(None)
df = pd.read_csv("data/BTCUSDT-trades-2023-04-01.zip", header=None, names=datatype.schema.names)

In [ ]:
df["time_sec"] = df["time"] / 3600000


In [ ]:
df.groupby("time_sec", as_index=False).agg({"price": "median"}).plot.scatter(x="time_sec", y="price")

In [ ]:
from  src.datatypes.btcusdt_eoh_option_trades import BtcUsdtEohOptionTrades
datatype = BtcUsdtEohOptionTrades(spark)
option_eoh = datatype.load(context.date_range)

In [ ]:
option_eoh.toPandas()

In [ ]:
spark.stop()